In [17]:
import open3d as o3d
import h5py
import os
import re

In [18]:
main_path="produzione_04032022"
num_points=250

In [19]:
#Print grid in readable format
def pretty_print(grid):
    for row in grid:
        print(row)

#from path of description file get the coupling grid, model data and 
#remove models with error
def process_file(path):
    with open(path,'r') as f:
        #don't consider first two lines
        f.readline()           #segmenti x x x
        f.readline()           #rotazioni x x x

        line=f.readline()      #numero pezzi x  

        #get number of fragments
        number=int(re.findall("\d+",line)[0])

        #coupling matrix
        grid=[]
        for i in range(number):
            line=f.readline()
            ret=re.findall('-1|0|1',line)
            grid.append(list(map(int,ret)))

        #model data
        model_names=[]
        non_valid_indeces=[]
        for m in range(number):
            f.readline()        #blank line
            name=f.readline()   #model name
            mesh=f.readline()   #mesh n
            f.readline()        #external n
            f.readline()        #internal n

            #if mesh=0 I don't consider the element
            mesh_n=int(mesh.rstrip().split(" ")[1]) 
            if mesh_n!=0: 
                #try create file name
                file_name= name.rstrip().replace(".","_")
                model_names.append(file_name)          
            else:
                #saving indices to remove later
                non_valid_indeces.append(m)

        #removing elements from grid
        #sorted in reverse to avoid wrong index
        for index in sorted(non_valid_indeces, reverse=True):
            #remove row
            del grid[index]
            #remove columns
            for row in grid:
                del row[index]
        
        return grid,model_names

In [20]:
#get a set of fragments (i.e. a subfolder)
def get_set(folder,verbose=True):
    folder_path=os.path.join(main_path,folder)
    models_file=[]

    #files are either text files or models
    for file in os.listdir(folder_path):
        if '.txt' in file:
            description_file=file
        elif '.stl' in file:
            #not used, they are not in the same order of the file
            models_file.append(file)

    #parsing description file
    full_description_file=os.path.join(folder_path,description_file)
    grid,model_names=process_file(full_description_file)

    #get path of models of current set
    model_prefix=folder.replace("generatedTest_","")
    complete_models_path=[]
    for i in range(len(model_names)):
        #e.g. 2021_11_29_10_00_40_Cube_001.stl
        correct_model_name=f"{model_prefix}_{model_names[i]}.stl"
        #saving only names, could be useful but now not used
        model_names[i]=correct_model_name

        complete_models_path.append(os.path.join(folder_path,correct_model_name))

    #create set and put into list of sets
    set={"models":complete_models_path,"grid":grid}
    
    if verbose:
        print("A set: \n")
        print("Models: ",set["models"])
        print("Grid: ")
        pretty_print(set["grid"])

    return set

In [21]:
def get_pointcloud(mesh,num_points):
    #pointcloud=mesh.sample_points_poisson_disk(num_points)
    pointcloud=mesh.sample_points_uniformly(num_points)

    center=pointcloud.get_center()
    pointcloud=pointcloud.translate(center*-1)

    return pointcloud


## Export or visualize pointcloud

In [ ]:

for folder in os.listdir(main_path):
    #check only folders, not files
    if '.' not in folder:
        fragment_set=get_set(folder,verbose=False)  #set of fragments of one model
        for path in fragment_set["models"]:
            mesh=o3d.io.read_triangle_mesh(path)
            pointcloud=get_pointcloud(mesh,num_points)
            normals=pointcloud.normals

            #visualize pointcloud and normals
            #o3d.visualization.draw_geometries([pointcloud])
            

            #export pointcloud in ply format
            #o3d.io.write_point_cloud("./pointclouds/data.ply", pointcloud)
            #o3d.visualization.draw_geometries([pointcloud])
        break